In [4]:
library(repr)
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [5]:
marketing_data <- read_delim("/home/jovyan/toy_ds_project/marketing_campaign.csv")
marketing_data    

Rows: 2240 Columns: 29
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (3): Education, Marital_Status, Dt_Customer
dbl (26): ID, Year_Birth, Income, Kidhome, Teenhome, Recency, MntWines, MntF...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,⋯,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5524,1957,Graduation,Single,58138,0,0,04-09-2012,58,635,⋯,7,0,0,0,0,0,0,3,11,1
2174,1954,Graduation,Single,46344,1,1,08-03-2014,38,11,⋯,5,0,0,0,0,0,0,3,11,0
4141,1965,Graduation,Together,71613,0,0,21-08-2013,26,426,⋯,4,0,0,0,0,0,0,3,11,0
6182,1984,Graduation,Together,26646,1,0,10-02-2014,26,11,⋯,6,0,0,0,0,0,0,3,11,0
5324,1981,PhD,Married,58293,1,0,19-01-2014,94,173,⋯,5,0,0,0,0,0,0,3,11,0
7446,1967,Master,Together,62513,0,1,09-09-2013,16,520,⋯,6,0,0,0,0,0,0,3,11,0
965,1971,Graduation,Divorced,55635,0,1,13-11-2012,34,235,⋯,6,0,0,0,0,0,0,3,11,0
6177,1985,PhD,Married,33454,1,0,08-05-2013,32,76,⋯,8,0,0,0,0,0,0,3,11,0
4855,1974,PhD,Together,30351,1,0,06-06-2013,19,14,⋯,9,0,0,0,0,0,0,3,11,1


**Tidying the Data:**

In [15]:
# selects yob, education, and income columns
marketing_data_tidy <- marketing_data |>
select(Year_Birth, Education, Income) 

# creates dataframe for people with only a basic degree
marketing_basic_ed <- marketing_data_tidy |> 
  filter(Education == "Basic")

# counts number of people with a basic degree
marketing_basic_ed_count <- summarize(marketing_basic_ed, n())
count <- pull(marketing_basic_ed_count, 1)

# creates dataframe for people with a bachelors degree
# (number of rows is equal to number of rows in marketing_basic_ed)
marketing_grad_ed <- marketing_data_tidy |> 
  filter(Education == "Graduation") |>
  head(count)

# rbind() stacks marketing__ed and marketing_grad_ed
marketing_data <- rbind(marketing_basic_ed, marketing_grad_ed)
marketing_data


Year_Birth,Education,Income
<dbl>,<chr>,<dbl>
1976,Basic,7500
1979,Basic,24594
1976,Basic,9548
1961,Basic,28249
1978,Basic,24882
1981,Basic,24480
1973,Basic,27190
1990,Basic,24279
1975,Basic,30801


**Summarizing the Data:**

In [16]:
# splits data into training and test set
marketing_data_split <- initial_split(marketing_data, prop = 0.75, strata = Education)
marketing_data_train <- training(marketing_data_split)
marketing_data_test <- testing(marketing_data_split) 

# summarizes training data
marketing_data_summarized <- marketing_data_train |>
    group_by(Education) |>
    summarize(Mean_Income = mean(Income, na.rm = TRUE))
marketing_data_summarized

Education,Mean_Income
<chr>,<dbl>
Basic,20309.45
Graduation,52785.06


**Visualizing the Data:**

Warning message:
“Removed 5 rows containing missing values (geom_point).”


In [17]:
# visualizes training data using scatterplot
marketing_plot <- marketing_data_train |> ggplot(aes(x = Year_Birth, y = Income, color = Education)) +
  geom_point() +
  labs(x = "Year of Birth", y = "Income")
marketing_plot